In [ ]:
# get ic decay
from get_data import get_data,bars
from singletrader.processors.cs_processor import CsWinzorize
from singletrader.shared.utility import save_pkl
from singletrader.factorlib import FactorEvaluation,summary_plot
import pandas as pd
import plotly.express as px
from plotly.figure_factory import create_table
import warnings
from tools.workflow import MultiFactorTesting
warnings.filterwarnings('ignore')





In [2]:
data = get_data().dropna() #获取处理好的数据
features = [_f for _f in data.columns if _f not in bars] + ['ep','circulating_market_cap']
bar_data = data[bars]
# bar_data.head()

# Tips
-   默认的极值处理方式均调整为为0.01/0.99放缩，前值为5倍标准差处理

In [3]:
liquidity_group = data.groupby(level=0).apply(lambda x:pd.qcut(x['amount3M'],3,labels=['low','medium','high'])).droplevel(0)
ep_group = data.groupby(level=0).apply(lambda x:pd.qcut(x['ep'],3,labels=['low','medium','high'])).droplevel(0)
high_liq =(liquidity_group=='high').astype(int)
print('high liquidity set\n',high_liq.head())

high liquidity set
 date        code       
2010-01-31  000001.XSHE    1
            000002.XSHE    1
            000004.XSHE    0
            000005.XSHE    1
            000006.XSHE    1
Name: amount3M, dtype: int32


# summary of all factors

- 非行业中性化(全市场)

In [4]:
# 非行业中性化(全市场)
fe = FactorEvaluation(bar_data=data[bars],factor_data=data[features],freq=12,winzorize={'method':'qtile','k':0.01},standardize=True,industry_neutralize=False)
fe.get_summary(add_shift=0,start_date='2010-01-01',end_date='2022-12-31',base='close',groups=10) # forward_return: 下月收盘-本月收盘
# fe.get_summary(add_shift=0,start_date='2010-01-01',end_date='2022-12-31',universe=high_liq,base='close-open') # forward_return: 下月收盘-下月开盘

start_date:2010-01-01 / end_date:2022-12-31


,skew,adjskew,distance,mom6x3,mom9x3,mom12x3,mom1M,mom2M,mom3M,mom6M,turnover3M,amount3M,stddev_diff,stddev,ep,circulating_market_cap
ic.mean,-0.0228,-0.0252,0.0015,0.0009,0.0069,0.0090,-0.0419,-0.0505,-0.0445,-0.0330,-0.0422,-0.0393,-0.0078,-0.0385,0.0110,-0.0123
ic.t-stats,-4.7346,-5.4992,0.1356,0.1148,0.7518,0.9592,-4.5227,-4.7153,-4.2887,-3.0849,-3.8007,-4.5036,-1.4424,-3.0906,1.5631,-1.3657
AnnRet_Long,-0.0631,-0.0609,0.0273,-0.0237,-0.0058,0.0096,-0.1463,-0.1504,-0.1493,-0.1172,-0.1189,-0.1245,-0.0772,-0.1316,-0.0026,-0.0712
AnnRet_short,0.0178,0.0397,0.0226,-0.0197,-0.0604,-0.0506,0.0257,0.0839,0.0669,0.0645,0.0795,0.1421,-0.0520,0.0493,0.0037,0.1681
AnnRet,-0.0809,-0.1006,0.0047,-0.0040,0.0546,0.0601,-0.1719,-0.2343,-0.2163,-0.1817,-0.1984,-0.2666,-0.0252,-0.1809,-0.0063,-0.2393
SR_Long,-1.2742,-1.3446,0.2515,-0.2369,-0.0554,0.0888,-1.3972,-1.2327,-1.1876,-0.9617,-1.0610,-1.0314,-1.0832,-1.1026,-0.0193,-0.4851
SR_Short,0.3605,0.8369,0.1808,-0.2561,-0.7717,-0.6149,0.2894,0.7633,0.6396,0.5825,0.5317,1.0871,-0.8670,0.3160,0.0492,1.5498
SR,-0.9711,-1.2559,0.0219,-0.0262,0.3344,0.3499,-1.0203,-1.1427,-1.0630,-0.8597,-0.8414,-1.2636,-0.2412,-0.7520,-0.0427,-1.0186
TO_Long,0.8872,0.8867,0.2880,0.5175,0.3709,0.3140,0.8619,0.6243,0.5150,0.3687,0.2387,0.1687,0.9489,0.6880,0.1019,0.0497
TO_Short,0.8816,0.8821,0.4891,0.5944,0.4543,0.3767,0.8966,0.7022,0.5939,0.4534,0.1791,0.2441,0.9482,0.4880,0.0876,0.1134


- 行业中性化(全市场)

In [ ]:
# 行业中性化(全市场)
fe = FactorEvaluation(bar_data=data[bars],factor_data=data[features],freq=12,winzorize={'method':'qtile','k':0.01},standardize=True,industry_neutralize=True)
fe.get_summary(add_shift=0,start_date='2010-01-01',end_date='2022-12-31',base='close',groups=10)

-   行业中性化因子（全市场） 2016~

In [ ]:
# 行业中性化因子（高流动性股票池） 2016~
fe.get_summary(add_shift=0,start_date='2016-01-01',end_date='2022-12-31',base='close',groups=10) # forward_return: 下月收盘-本月收盘

-   行业中性化因子（高流动性股票池）

In [ ]:
# 行业中性化因子（高流动性股票池）
fe = FactorEvaluation(bar_data=data[bars],factor_data=data[features],freq=12,winzorize={'method':'qtile','k':0.01},standardize=True,industry_neutralize=True)
fe.get_summary(add_shift=0,start_date='2010-01-01',universe=high_liq,end_date='2022-12-31',base='close',groups=10) # forward_return: 下月收盘-本月收盘

# double sort (mom12x3 - turnover3M) universe = high liquidity

-   row data(3x3)

In [ ]:
fe = FactorEvaluation(bar_data=data[bars],factor_data=data[features],freq=12,winzorize={'method':'qtile','k':0.01},standardize=True,industry_neutralize=False)
mft = MultiFactorTesting(bar_data=data.dropna()[bars],market_cap_col='circulating_market_cap',universe=high_liq,factor_data=fe.factor_data[['mom12x3','turnover3M']],add_shift=0)
summary = mft.summary(quantiles=3)
for i in list(summary.keys())[:2]+['avg_number_of_stocks']:print(i,'\n',summary[i])

-   raw data(5x5) 

In [ ]:
summary = mft.summary(quantiles=5)
for i in list(summary.keys())[:2]+['avg_number_of_stocks']:print(i,'\n',summary[i])

-   industry nutrualized data(3x3)


In [ ]:
fe = FactorEvaluation(bar_data=data[bars],factor_data=data[features],freq=12,winzorize={'method':'qtile','k':0.01},standardize=True,industry_neutralize=True)
mft = MultiFactorTesting(bar_data=data.dropna()[bars],market_cap_col='circulating_market_cap',universe=high_liq,factor_data=fe.factor_data[['mom12x3_GN','turnover3M_GN']],add_shift=0)
summary = mft.summary(quantiles=3)
for i in list(summary.keys())[:2]+['avg_number_of_stocks']:print(i,'\n',summary[i])

-  industry nutrualized data(5x5)


In [ ]:
summary = mft.summary(quantiles=5)
for i in list(summary.keys())[:2]+['avg_number_of_stocks']:print(i,'\n',summary[i])

# details of mom12x3_GN
收益曲线和表格指标，均是针对超额收益，benchmark用的是equal weighted

In [ ]:
factor='mom12x3_GN'
report = fe.get_factor_detail_report(factor=factor,universe=high_liq,add_shift=0,base='close',start_date='2010-01-01',end_date='2022-12-31',total=False,excess_return=True,holding_period=1)
summary_plot(report)


In [ ]:
group_ic = fe.factor_ana(factor=factor,ep_group=ep_group,liquidity_group=liquidity_group,add_shift=0,base='close')
bar = px.bar(group_ic['ic.mean'].reset_index(),x='level_1',y='ic.mean',color='level_0',barmode='group',title='ic of different ep & liquidity')
bar.show()



note: mom12x3在流动性差的股票体现反转特性，在流动性高的股票呈现趋势特性